<a href="https://colab.research.google.com/github/DiegoLangreo7/EII-RI/blob/main/Ejercicio2_RI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Ej. 2. Experimentación con variantes de BM25


En este segundo ejercicio, se nos pide desarrollar un código que genere “rondas” de resultados con 100 documentos (solo los identificadores) para todas las consultas de la colección. Dichas rondas deben ejecutarse considerando todas las combinaciones posibles de parámetros.

Empezamos preparando los archivos necesarios, como hicimos en el ejercicio anterior:

In [ ]:
#descargamos paquete bm25s
!pip install bm25s[full]
#descargamos las librerias que usaremos
import bm25s
import Stemmer
import json
#descargamos la coleccion y las query
!gdown 19pzNFYIch8rj9d3kyq-171V8vRa_wLBC
!unzip -o trec-covid-RI.zip
#parseamos la coleccion
with open("corpus.jsonl", "r", encoding="utf-8") as f:
    corpus_content = []
    for line in f:
        corpus_content.append(json.loads(line))
#indexamos la coleccion
corpus_verbatim = list()
corpus_plaintext = list()
for entry in corpus_content:
  document = {"id": entry["_id"], "title": entry["title"].lower(),
  "text": entry["text"].lower()}
  corpus_verbatim.append(document)
  corpus_plaintext.append(entry["text"].lower())

Ya tenemos la coleccion preparada para la realización del ejercicio.

Ahora, usaremos este método extraido de las prácticas de laboratorio, que nos permitirá enviar las consultas y obtener los resultados asociados a ellas, para
posteriormente calcular el rendimiento:


In [ ]:
def submit_queries_and_get_run(queries, stemmer, retriever, stopwords, max_results=100):
    # Inicializa un diccionario para almacenar los resultados de cada consulta
    run = {}

    # Itera sobre cada consulta en la lista de consultas
    for query in queries:
        # Obtiene el ID único de la consulta
        query_id = query["_id"]

        # Convierte el texto de la consulta a minúsculas
        query_string = query["text"].lower()

        # Tokeniza la consulta utilizando las stopwords y el stemmer proporcionados
        query_tokenized = bm25s.tokenize(
            query_string,
            stopwords=stopwords,
            stemmer=stemmer,
            show_progress=False
        )

        # Recupera los documentos relevantes usando el índice BM25
        results = retriever.retrieve(
            query_tokenized,
            corpus=retriever.corpus,
            k=max_results,
            return_as="tuple",
            show_progress=False
        )

        # Obtiene los documentos recuperados y sus scores de relevancia
        returned_documents = results.documents[0]

        # Inicializa una lista para almacenar los IDs de los documentos recuperados
        returned_ids = []
        for i in range(len(returned_documents)):
            # Agrega el ID de cada documento recuperado a la lista
            returned_ids.append(str(returned_documents[i]["id"]))

        # Asocia la lista de IDs recuperados con el ID de la consulta en el diccionario
        run[query_id] = returned_ids

    # Devuelve el diccionario con los resultados de todas las consultas
    return run


Preparamos las querys que vamos a utilizar con este otro metodo, tambien extraido de las practicas de laboratorio:

In [ ]:
queries = []

with open("queries.jsonl", "r", encoding="utf-8") as f:
    for line in f:
        queries.append(json.loads(line))

Creamos un metodo para poder descargar los archivos en disco, donde apareceran las metricas y el restulado de la run correspondiente:

In [ ]:
def save_run_to_disk(run, bm25_flavor, stopwords, stemming, metrics, max_results=100):
    stopwords_str = "Stopwords" if stopwords else "NON-stopwords"
    stemming_str = "Stemming" if stemming else "NON-stemming"

    filename = f"{bm25_flavor}-{stopwords_str}-{stemming_str}.json"

    # Crear una estructura que combine los resultados del run y las métricas
    run_data = {
        "bm25_flavor": bm25_flavor,
        "stopwords": stopwords_str,
        "stemming": stemming_str,
        "max_results": max_results,
        "metrics": {
            "macro_precision": metrics["macro_precision"],
            "macro_recall": metrics["macro_recall"],
            "macro_f1": metrics["macro_f1"],
            "micro_precision": metrics["micro_precision"],
            "micro_recall": metrics["micro_recall"],
            "micro_f1": metrics["micro_f1"]
        },
        "run": run
    }

    # Guardar los datos combinados en un archivo JSON
    with open(filename, 'w', encoding='utf-8') as f:
        json.dump(run_data, f, ensure_ascii=False, indent=4)

Ahora que ya tenemos la coleccion indexada, las queries y el metodo para ir experimentando, preparamos el stemmer, tokenizamos para cada caso, e indexandolo antes de llamar a la funcion "submit_queries_and_get_run" que realizara la busqueda. Luego estos resultados se guardaran en disco para aportar a la entrega, y en un array para su posterior uso.


In [ ]:
# Stemmer para cuando queramos usarlo (cuando no, ponemos directamente NONE)
stemmer = Stemmer.Stemmer("english")

# Lista para almacenar las diferentes configuraciones de ejecuciones
runs = []

# Iterar por todas las configuraciones
for bm25_config in ["lucene", "robertson", "atire", "bm25+", "bm25l"]:
    for stopwords_config in [None, "en"]:  # None es sin stopwords, "en" es con stopwords
        for stemming_config in [None, stemmer]:  # None es sin stemming, stemmer es con stemming
            # Configurar BM25
            bm25_flavor = bm25_config
            idf_flavor = bm25_config

            # Tokenizar colección
            corpus_tokenized = bm25s.tokenize(corpus_plaintext,stopwords=stopwords_config,
                                              stemmer=stemming_config,show_progress=True)

            # Indexar colección tokenizada
            retriever = bm25s.BM25(corpus=corpus_verbatim,method=bm25_flavor,
                idf_method=idf_flavor)
            retriever.index(corpus_tokenized, show_progress=True)

            # Ejecutar consultas y obtener resultados
            run = submit_queries_and_get_run(queries, stemming_config, retriever, stopwords_config, max_results=100)

            # Almacenar resultados para uso posterior
            runs.append(run)

Una vez tenemos almacenados las colecciones indexadas de las distintas formas (en total 20), vamos a proceder a las rondas de busqueda. Para ello vamos a hacer uso del método  y de las queries antes cargadas:

In [ ]:
# Función adaptada para calcular precision, recall y F1

def compute_precision_recall_f1(run, relevance_judgements):
    precision_values = []
    recall_values = []
    f1_values = []

    global_retrieved = 0
    global_relevant = 0
    global_retrieved_and_relevant = 0

    for query_id in run.keys():
        retrieved_results = run[query_id]
        relevant_results = relevance_judgements.get(query_id, [])
        relevant_and_retrieved = set(retrieved_results) & set(relevant_results)

        global_retrieved += len(retrieved_results)
        global_relevant += len(relevant_results)
        global_retrieved_and_relevant += len(relevant_and_retrieved)

        precision = len(relevant_and_retrieved) / len(retrieved_results) if len(retrieved_results) > 0 else 0
        recall = len(relevant_and_retrieved) / len(relevant_results) if len(relevant_results) > 0 else 0

        if (precision + recall) > 0:
            f1 = 2 * (precision * recall) / (precision + recall)
            f1_values.append(f1)

        precision_values.append(precision)
        recall_values.append(recall)

    macro_average_precision = sum(precision_values) / len(precision_values) if precision_values else 0
    macro_average_recall = sum(recall_values) / len(recall_values) if recall_values else 0
    macro_average_f1 = sum(f1_values) / len(f1_values) if f1_values else 0

    micro_average_precision = global_retrieved_and_relevant / global_retrieved if global_retrieved > 0 else 0
    micro_average_recall = global_retrieved_and_relevant / global_relevant if global_relevant > 0 else 0
    micro_average_f1 = (2 * (micro_average_precision * micro_average_recall) /
                        (micro_average_precision + micro_average_recall)) if (micro_average_precision + micro_average_recall) > 0 else 0

    return {
        "macro_precision": macro_average_precision,
        "macro_recall": macro_average_recall,
        "macro_f1": macro_average_f1,
        "micro_precision": micro_average_precision,
        "micro_recall": micro_average_recall,
        "micro_f1": micro_average_f1
    }

# Cargar juicios de relevancia desde qrels.tsv
relevance_judgements = {}
with open("qrels.tsv", "r", encoding="utf-8") as f:
    next(f)  # Saltar la cabecera
    for line in f:
        query_id, corpus_id, score = line.strip().split("\t")
        if int(score) > 0:  # Considerar solo documentos relevantes (score > 0)
            if query_id not in relevance_judgements:
                relevance_judgements[query_id] = []
            relevance_judgements[query_id].append(corpus_id)

# Esto es con el fin de identificar cada resultado
bm25_configs = ["lucene", "robertson", "atire", "bm25+", "bm25l"]
stopwords_configs = [None, "en"]
stemming_configs = [None, Stemmer.Stemmer("english")]
config_index = 0
# Ahora si lo calculamos
for bm25_config in bm25_configs:
    for stopwords_config in stopwords_configs:
        for stemming_config in stemming_configs:
            print(f"Evaluando configuración {config_index + 1}: BM25={bm25_config}, Stopwords={'Yes' if stopwords_config else 'No'}, Stemming={'Yes' if stemming_config else 'No'}")
            metrics = compute_precision_recall_f1(runs[config_index], relevance_judgements)
            print(f"Macro-averaged Precision: {metrics['macro_precision']:.3f}")
            print(f"Macro-averaged Recall: {metrics['macro_recall']:.3f}")
            print(f"Macro-averaged F1: {metrics['macro_f1']:.3f}\n")
            print(f"Micro-averaged Precision: {metrics['micro_precision']:.3f}")
            print(f"Micro-averaged Recall: {metrics['micro_recall']:.3f}")
            print(f"Micro-averaged F1: {metrics['micro_f1']:.3f}\n")

            # Guardar el run y las métricas asociadas en disco
            save_run_to_disk(
                 run=runs[config_index],
                 bm25_flavor=bm25_config,
                 stopwords=stopwords_config,
                 stemming=stemming_config,
                 metrics=metrics
            )

            config_index += 1

En base a los resultados obtenidos, el mejor promediado es la configuracion 19, es decir, BM25=robertson / Stopwords=No / Stemming=Yes
